# **Import**

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# !pip install keras-tcn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tcn import TCN

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import mean_absolute_error, mean_squared_error

# **Data Load**

In [ ]:
cd /content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Data

/content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Data


In [ ]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = [f's_{i + 1}' for i in range(21)]
col_names = index_names + setting_names + sensor_names

train_df = pd.read_csv('./train_FD001.csv')

In [ ]:
max_cycle = train_df.groupby('unit_number')['time_cycles'].max().reset_index()
max_cycle.columns = ['unit_number', 'max_cycle']

train_df = train_df.merge(max_cycle, on='unit_number', how='left')

train_df['RUL'] = train_df['max_cycle'] - train_df['time_cycles']

In [ ]:
test_df = pd.read_csv('./test_FD001.csv')
rul_df = pd.read_csv('./RUL_FD001.csv')

In [ ]:
results = []

using_sensors = [
    's_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12',
    's_13', 's_14', 's_15', 's_17', 's_20', 's_21'
]

## Raw Data

In [ ]:
def make_train_sequence_data(df, seq_len):
    x, y = [], []

    for unit in df['unit_number'].unique():
        unit_df = df[df['unit_number'] == unit]
        sensor_values = unit_df[using_sensors].values
        rul_values = unit_df['RUL'].values

        for i in range(len(unit_df) - seq_len + 1):
            x.append(sensor_values[i:i + seq_len])
            y.append(rul_values[i + seq_len - 1])

    return np.array(x), np.array(y)

def make_test_sequence_data(df, seq_len):
    x = []
    unit_list = df['unit_number'].unique()

    for unit in unit_list:
        unit_df = df[df['unit_number'] == unit]
        values = unit_df[using_sensors].values

        if len(unit_df) >= seq_len:
            x.append(values[-seq_len:])
        else:
            pad = np.repeat(values[-1:], seq_len - len(unit_df), axis=0)
            x.append(np.vstack([pad, values]))

    return np.array(x)

In [ ]:
def build_lstm(seq_len, n_features):
    return Sequential([
        LSTM(64, input_shape=(seq_len, n_features)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])

def build_tcn(seq_len, n_features):
    return Sequential([
        TCN(64, input_shape=(seq_len, n_features)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])

In [ ]:
raw_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

for seq_len in [30, 40, 50]:
    for scaler_name, scaler in scaler_dict.items():
        scaled_train_df = train_df.copy()
        scaled_test_df = test_df.copy()

        if scaler:
            x_train_df = scaled_train_df[using_sensors]
            x_test_df = scaled_test_df[using_sensors]

            scaler.fit(x_train_df)

            scaled_train_df[using_sensors] = scaler.transform(x_train_df)
            scaled_test_df[using_sensors] = scaler.transform(x_test_df)

        train_x, train_y = make_train_sequence_data(scaled_train_df, seq_len)
        test_x = make_test_sequence_data(scaled_test_df, seq_len)

        model_dict = {
            'LSTM': build_lstm(seq_len, len(using_sensors)),
            'TCN': build_tcn(seq_len, len(using_sensors))
        }
        es = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
        for model_name, model in model_dict.items():
            model.compile(optimizer='adam', loss='mse', metrics=['mae'])
            model.fit(
                train_x, train_y,
                epochs=500, batch_size=32,
                validation_split=0.2,
                callbacks=[es],
                verbose=0
            )

            pred = model.predict(test_x).ravel()
            true = rul_df['RUL'].copy().values.ravel()

            mae = mean_absolute_error(true, pred)
            rmse = mean_squared_error(true, pred)**0.5

            print(f'Seq: {seq_len}, Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

            raw_results.append({
                'Data': f'Raw - {seq_len}',
                'Model': model_name,
                'Scaler': scaler_name,
                'MAE': mae,
                'RMSE': rmse
            })
results.extend(raw_results)

Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 6.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Seq: 30, Model: LSTM, Scaler: None, MAE: 36.0945930480957, RMSE: 44.44669604214131
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 6.
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 441ms/step
Seq: 30, Model: TCN, Scaler: None, MAE: 36.57931900024414, RMSE: 45.66965922182363
Epoch 19: early stopping
Restoring model weights from the end of the best epoch: 14.


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Seq: 30, Model: LSTM, Scaler: MinMaxScaler, MAE: 18.107688903808594, RMSE: 25.649317967677643
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 5.
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step
Seq: 30, Model: TCN, Scaler: MinMaxScaler, MAE: 17.973255157470703, RMSE: 24.829275461073667
Epoch 9: early stopping
Restoring model weights from the end of the best epoch: 4.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Seq: 30, Model: LSTM, Scaler: StandardScaler, MAE: 16.403640747070312, RMSE: 23.273571248356873
Epoch 9: early stopping
Restoring model weights from the end of the best epoch: 4.
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 348ms/step
Seq: 30, Model: TCN, Scaler: StandardScaler, MAE: 24.607437133789062, RMSE: 33.27740588128077
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 3.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Seq: 30, Model: LSTM, Scaler: RobustScaler, MAE: 17.555830001831055, RMSE: 24.2707015878488
Epoch 

In [ ]:
pd.DataFrame(raw_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
13,Raw - 40,TCN,StandardScaler,15.465040,21.947829
14,Raw - 40,LSTM,RobustScaler,15.528936,23.459107
12,Raw - 40,LSTM,StandardScaler,15.611986,23.833527
22,Raw - 50,LSTM,RobustScaler,16.021315,24.198052
10,Raw - 40,LSTM,MinMaxScaler,16.160391,23.792680
4,Raw - 30,LSTM,StandardScaler,16.403641,23.273571
11,Raw - 40,TCN,MinMaxScaler,16.836288,26.226095
15,Raw - 40,TCN,RobustScaler,16.915634,24.125678
23,Raw - 50,TCN,RobustScaler,17.266756,23.417641
20,Raw - 50,LSTM,StandardScaler,17.286097,25.298172


In [ ]:
raw_clipping_results = []

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

for seq_len in [30, 40, 50]:
    for scaler_name, scaler in scaler_dict.items():
        scaled_train_df = train_df.copy()
        scaled_test_df = test_df.copy()

        scaled_train_df.loc[scaled_train_df['RUL'] > 125, 'RUL'] = 125

        if scaler:
            x_train_df = scaled_train_df[using_sensors]
            x_test_df = scaled_test_df[using_sensors]

            scaler.fit(x_train_df)

            scaled_train_df[using_sensors] = scaler.transform(x_train_df)
            scaled_test_df[using_sensors] = scaler.transform(x_test_df)

        train_x, train_y = make_train_sequence_data(scaled_train_df, seq_len)
        test_x = make_test_sequence_data(scaled_test_df, seq_len)

        model_dict = {
            'LSTM': build_lstm(seq_len, len(using_sensors)),
            'TCN': build_tcn(seq_len, len(using_sensors))
        }
        es = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
        for model_name, model in model_dict.items():
            model.compile(optimizer='adam', loss='mse', metrics=['mae'])
            model.fit(
                train_x, train_y,
                epochs=500, batch_size=32,
                validation_split=0.2,
                callbacks=[es],
                verbose=0
            )

            pred = model.predict(test_x).ravel()
            true = rul_df['RUL'].copy()
            true.loc[true > 125] = 125
            true = true.values.ravel()

            mae = mean_absolute_error(true, pred)
            rmse = mean_squared_error(true, pred)**0.5

            print(f'Seq: {seq_len}, Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

            raw_clipping_results.append({
                'Data': f'Raw (RUL Clipping) - {seq_len}',
                'Model': model_name,
                'Scaler': scaler_name,
                'MAE': mae,
                'RMSE': rmse
            })
results.extend(raw_clipping_results)

Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Seq: 30, Model: LSTM, Scaler: None, MAE: 35.053443908691406, RMSE: 40.59950522135399
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 13.
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 372ms/step
Seq: 30, Model: TCN, Scaler: None, MAE: 35.988643646240234, RMSE: 39.79457443291579
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 11.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Seq: 30, Model: LSTM, Scaler: MinMaxScaler, MAE: 10.299077987670898, RMSE: 14.282277811072582
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 3.
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 377ms/step
Seq: 30, Model: TCN, Scaler: MinMaxScaler, MAE: 11.083958625793457, RMSE: 15.089146008654957
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 5.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Seq: 30, Model: LSTM, Scaler: S

In [ ]:
pd.DataFrame(raw_clipping_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
18,Raw (RUL Clipping) - 50,LSTM,MinMaxScaler,9.087166,12.664556
10,Raw (RUL Clipping) - 40,LSTM,MinMaxScaler,9.435818,13.095552
12,Raw (RUL Clipping) - 40,LSTM,StandardScaler,9.459228,13.203378
6,Raw (RUL Clipping) - 30,LSTM,RobustScaler,9.557278,13.141451
14,Raw (RUL Clipping) - 40,LSTM,RobustScaler,9.585414,13.079066
20,Raw (RUL Clipping) - 50,LSTM,StandardScaler,9.633820,12.773816
22,Raw (RUL Clipping) - 50,LSTM,RobustScaler,9.735457,13.686190
2,Raw (RUL Clipping) - 30,LSTM,MinMaxScaler,10.299078,14.282278
19,Raw (RUL Clipping) - 50,TCN,MinMaxScaler,10.446429,13.955136
3,Raw (RUL Clipping) - 30,TCN,MinMaxScaler,11.083959,15.089146


## **Cumulative Stat**

In [ ]:
def make_cum_stat(df, is_train=True):
    tmp_df = df.copy()
    stat_df = []

    for unit in tmp_df['unit_number'].unique():
        unit_data = tmp_df[tmp_df['unit_number'] == unit]
        for t in range(2, len(unit_data) + 1):  # 최소 2개 시점부터
            sub_data = unit_data.iloc[:t]  # 1~t
            features = {'unit_number': unit, 'time_cycles': sub_data['time_cycles'].iloc[-1]}

            for sensor in using_sensors:
                features[f'{sensor}_mean'] = sub_data[sensor].mean()
                features[f'{sensor}_std'] = sub_data[sensor].std()
                features[f'{sensor}_min'] = sub_data[sensor].min()
                features[f'{sensor}_max'] = sub_data[sensor].max()
                features[f'{sensor}_last'] = sub_data[sensor].iloc[-1]
                features[f'{sensor}_median'] = sub_data[sensor].median()
                # trend는 최소 2개 이상 시점에서만 계산 가능
                features[f'{sensor}_trend'] = np.polyfit(sub_data['time_cycles'], sub_data[sensor], 1)[0]

            if is_train:
                features['RUL'] = sub_data['RUL'].iloc[-1]

            stat_df.append(features)

    return pd.DataFrame(stat_df)

In [ ]:
def make_cum_train_sequence_data(df, seq_len):
    x, y = [], []

    for unit in df['unit_number'].unique():
        unit_df = df[df['unit_number'] == unit]
        sensor_values = unit_df[using_stats].values
        rul_values = unit_df['RUL'].values

        for i in range(len(unit_df) - seq_len + 1):
            x.append(sensor_values[i:i + seq_len])
            y.append(rul_values[i + seq_len - 1])

    return np.array(x), np.array(y)

def make_cum_test_sequence_data(df, seq_len):
    x = []
    unit_list = df['unit_number'].unique()

    for unit in unit_list:
        unit_df = df[df['unit_number'] == unit]
        values = unit_df[using_stats].values

        if len(unit_df) >= seq_len:
            x.append(values[-seq_len:])
        else:
            pad = np.repeat(values[-1:], seq_len - len(unit_df), axis=0)
            x.append(np.vstack([pad, values]))

    return np.array(x)

In [ ]:
cum_stat_train_df = make_cum_stat(train_df, is_train=True)
cum_stat_test_df = make_cum_stat(test_df, is_train=False)

In [ ]:
cum_stat_results = []

using_stats = [col for col in cum_stat_train_df.columns if col.startswith('s')]

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

for seq_len in [30, 40, 50]:
    for scaler_name, scaler in scaler_dict.items():
        scaled_train_df = cum_stat_train_df.copy()
        scaled_test_df = cum_stat_test_df.copy()

        if scaler:
            x_train_df = scaled_train_df[using_stats]
            x_test_df = scaled_test_df[using_stats]

            scaler.fit(x_train_df)

            scaled_train_df[using_stats] = scaler.transform(x_train_df)
            scaled_test_df[using_stats] = scaler.transform(x_test_df)

        train_x, train_y = make_cum_train_sequence_data(scaled_train_df, seq_len)
        test_x = make_cum_test_sequence_data(scaled_test_df, seq_len)

        model_dict = {
            'LSTM': build_lstm(seq_len, len(using_stats)),
            'TCN': build_tcn(seq_len, len(using_stats))
        }
        es = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
        for model_name, model in model_dict.items():
            model.compile(optimizer='adam', loss='mse', metrics=['mae'])
            model.fit(
                train_x, train_y,
                epochs=500, batch_size=32,
                validation_split=0.2,
                callbacks=[es],
                verbose=0
            )

            pred = model.predict(test_x).ravel()
            true = rul_df['RUL'].copy().values.ravel()

            mae = mean_absolute_error(true, pred)
            rmse = mean_squared_error(true, pred)**0.5

            print(f'Seq: {seq_len}, Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

            cum_stat_results.append({
                'Data': f'Cumulative Stat - {seq_len}',
                'Model': model_name,
                'Scaler': scaler_name,
                'MAE': mae,
                'RMSE': rmse
            })
results.extend(cum_stat_results)

Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 8.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Seq: 30, Model: LSTM, Scaler: None, MAE: 36.17548751831055, RMSE: 44.69029711525758
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 5.
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 417ms/step
Seq: 30, Model: TCN, Scaler: None, MAE: 36.714805603027344, RMSE: 45.955321798051855
Epoch 9: early stopping
Restoring model weights from the end of the best epoch: 4.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Seq: 30, Model: LSTM, Scaler: MinMaxScaler, MAE: 17.18834686279297, RMSE: 24.397703269051526
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 406ms/step
Seq: 30, Model: TCN, Scaler: MinMaxScaler, MAE: 17.73922348022461, RMSE: 23.748123897939287
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Seq: 30, Model: LSTM, Scaler: Standar

In [ ]:
pd.DataFrame(cum_stat_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
22,Cumulative Stat - 50,LSTM,RobustScaler,11.631515,18.200501
14,Cumulative Stat - 40,LSTM,RobustScaler,13.153882,20.197823
6,Cumulative Stat - 30,LSTM,RobustScaler,14.701905,23.422095
12,Cumulative Stat - 40,LSTM,StandardScaler,15.150839,21.247225
20,Cumulative Stat - 50,LSTM,StandardScaler,15.379727,23.310453
4,Cumulative Stat - 30,LSTM,StandardScaler,15.428389,21.626723
19,Cumulative Stat - 50,TCN,MinMaxScaler,15.528831,22.683798
11,Cumulative Stat - 40,TCN,MinMaxScaler,15.560795,23.101071
2,Cumulative Stat - 30,LSTM,MinMaxScaler,17.188347,24.397703
3,Cumulative Stat - 30,TCN,MinMaxScaler,17.739223,23.748124


In [ ]:
# RUL Clipping O
cum_stat_rul_clipping_results = []

using_stats = [col for col in cum_stat_train_df.columns if col.startswith('s')]

scaler_dict = {
    'None': None,
    'MinMaxScaler': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler()
}

for seq_len in [30, 40, 50]:
    for scaler_name, scaler in scaler_dict.items():
        scaled_train_df = cum_stat_train_df.copy()
        scaled_test_df = cum_stat_test_df.copy()

        scaled_train_df.loc[scaled_train_df['RUL'] > 125, 'RUL'] = 125

        if scaler:
            x_train_df = scaled_train_df[using_stats]
            x_test_df = scaled_test_df[using_stats]

            scaler.fit(x_train_df)

            scaled_train_df[using_stats] = scaler.transform(x_train_df)
            scaled_test_df[using_stats] = scaler.transform(x_test_df)

        train_x, train_y = make_cum_train_sequence_data(scaled_train_df, seq_len)
        test_x = make_cum_test_sequence_data(scaled_test_df, seq_len)

        model_dict = {
            'LSTM': build_lstm(seq_len, len(using_stats)),
            'TCN': build_tcn(seq_len, len(using_stats))
        }
        es = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
        for model_name, model in model_dict.items():
            model.compile(optimizer='adam', loss='mse', metrics=['mae'])
            model.fit(
                train_x, train_y,
                epochs=500, batch_size=32,
                validation_split=0.2,
                callbacks=[es],
                verbose=0
            )

            pred = model.predict(test_x).ravel()
            true = rul_df['RUL'].copy()
            true.loc[true > 125] = 125
            true = true.values.ravel()

            mae = mean_absolute_error(true, pred)
            rmse = mean_squared_error(true, pred)**0.5

            print(f'Seq: {seq_len}, Model: {model_name}, Scaler: {scaler_name}, MAE: {mae}, RMSE: {rmse}')

            cum_stat_rul_clipping_results.append({
                'Data': f'Cumulative Stat (RUL Clipping) - {seq_len}',
                'Model': model_name,
                'Scaler': scaler_name,
                'MAE': mae,
                'RMSE': rmse
            })
results.extend(cum_stat_rul_clipping_results)

Epoch 9: early stopping
Restoring model weights from the end of the best epoch: 4.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Seq: 30, Model: LSTM, Scaler: None, MAE: 35.146705627441406, RMSE: 40.46031222363929
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 377ms/step
Seq: 30, Model: TCN, Scaler: None, MAE: 34.48139953613281, RMSE: 40.79366579440137
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Seq: 30, Model: LSTM, Scaler: MinMaxScaler, MAE: 9.117430686950684, RMSE: 12.710190800901161
Epoch 12: early stopping
Restoring model weights from the end of the best epoch: 7.
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 369ms/step
Seq: 30, Model: TCN, Scaler: MinMaxScaler, MAE: 10.717375755310059, RMSE: 14.773050175052184
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 2.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Seq: 30, Model: LSTM, Scaler: Standar

In [ ]:
pd.DataFrame(cum_stat_rul_clipping_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
22,Cumulative Stat (RUL Clipping) - 50,LSTM,RobustScaler,7.834088,12.397012
18,Cumulative Stat (RUL Clipping) - 50,LSTM,MinMaxScaler,8.406834,12.200348
6,Cumulative Stat (RUL Clipping) - 30,LSTM,RobustScaler,8.521237,12.822533
14,Cumulative Stat (RUL Clipping) - 40,LSTM,RobustScaler,8.550470,13.436869
2,Cumulative Stat (RUL Clipping) - 30,LSTM,MinMaxScaler,9.117431,12.710191
12,Cumulative Stat (RUL Clipping) - 40,LSTM,StandardScaler,9.501124,13.781271
20,Cumulative Stat (RUL Clipping) - 50,LSTM,StandardScaler,9.631374,14.526542
11,Cumulative Stat (RUL Clipping) - 40,TCN,MinMaxScaler,9.922146,13.160652
4,Cumulative Stat (RUL Clipping) - 30,LSTM,StandardScaler,10.311346,14.200599
19,Cumulative Stat (RUL Clipping) - 50,TCN,MinMaxScaler,10.689284,14.222494


# **Result**

In [ ]:
result = pd.DataFrame(results)
result.to_csv('/content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Results/Sequence_Regression.csv', index=False)

In [ ]:
result = result.sort_values(by='MAE')
result.head()

,Data,Model,Scaler,MAE,RMSE
94,Cumulative Stat (RUL Clipping) - 50,LSTM,RobustScaler,7.834088,12.397012
90,Cumulative Stat (RUL Clipping) - 50,LSTM,MinMaxScaler,8.406834,12.200348
78,Cumulative Stat (RUL Clipping) - 30,LSTM,RobustScaler,8.521237,12.822533
86,Cumulative Stat (RUL Clipping) - 40,LSTM,RobustScaler,8.550470,13.436869
42,Raw (RUL Clipping) - 50,LSTM,MinMaxScaler,9.087166,12.664556
